Stacked RNN... Softmax 연결하기

In [1]:
# Lab 12 Character Sequence Softmax only
import tensorflow as tf
###Disable Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import logging
tf.get_logger().setLevel(logging.ERROR)
#import sys
#original_stdout = sys.stdout
#sys.stdout = None
#import warnings
#warnings.filterwarnings("ignore")

import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

# flatten the data (ignore batches for now). No effect if the batch size is 1
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
X_for_softmax = tf.reshape(X_one_hot, [-1, rnn_hidden_size]) #RNN 출력을 flatten해줌

# softmax layer (rnn_hidden_size -> num_classes)
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes]) 
    #softmax의 입력은 hidden_size, 출력은 classes에 맞춘다. 데이터플로우 차트를 확인
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# expend the data (revive the batches)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

# Compute sequence cost/loss
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights) #output에는 activation function을 거치지 않은 logits을 입력해줘야 좋은 성능을 갖는다. RNN에서 activation function이라 하면 tanh가 포함된 cell 함수들이 있다.
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss:", l, "Prediction:", ''.join(result_str))

'''
0 loss: 2.29513 Prediction: yu yny y y oyny
1 loss: 2.10156 Prediction: yu ynu y y oynu
2 loss: 1.92344 Prediction: yu you y u  you

..

2997 loss: 0.277323 Prediction: yf you yant you
2998 loss: 0.277323 Prediction: yf you yant you
2999 loss: 0.277323 Prediction: yf you yant you
'''


0 loss: 2.50339 Prediction: u  u   u  unu  
1 loss: 2.3071556 Prediction: u  u   u   nu  
2 loss: 2.1235278 Prediction:                
3 loss: 1.9532908 Prediction:      u        u
4 loss: 1.7954704 Prediction: y  you y    you
5 loss: 1.6486022 Prediction: y  you y  t you
6 loss: 1.5115441 Prediction: y  you y  t you
7 loss: 1.3837068 Prediction: yf you y  t you
8 loss: 1.2648406 Prediction: yf you ya t you
9 loss: 1.1547207 Prediction: yf you yant you
10 loss: 1.0531803 Prediction: yf you yant you
11 loss: 0.9601961 Prediction: yf you yant you
12 loss: 0.8757897 Prediction: yf you yant you
13 loss: 0.7999348 Prediction: yf you yant you
14 loss: 0.7324983 Prediction: yf you yant you
15 loss: 0.6732033 Prediction: yf you yant you
16 loss: 0.62162167 Prediction: yf you yant you
17 loss: 0.5771838 Prediction: yf you yant you
18 loss: 0.53920096 Prediction: yf you yant you
19 loss: 0.5069002 Prediction: yf you yant you
20 loss: 0.4794752 Prediction: yf you yant you
21 loss: 0.45614478 Pre

'\n0 loss: 2.29513 Prediction: yu yny y y oyny\n1 loss: 2.10156 Prediction: yu ynu y y oynu\n2 loss: 1.92344 Prediction: yu you y u  you\n\n..\n\n2997 loss: 0.277323 Prediction: yf you yant you\n2998 loss: 0.277323 Prediction: yf you yant you\n2999 loss: 0.277323 Prediction: yf you yant you\n'